<a href="https://colab.research.google.com/github/AAhmadS/NLP_HW2/blob/main/Final_Copy_of_NLP_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/NLP

/content/gdrive/.shortcut-targets-by-id/1Ikn75GANDVFyb0klJe4-EBQJikt4_Ob_/NLP


##Extracting essential data



###Essential packages

In [46]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [47]:
#installing the necessary libraries
!pip install -q hazm
!pip install -q dadmatools

In [5]:
#importing the necessary libraries
from __future__ import unicode_literals
import hazm

import dadmatools.pipeline.language as language

import json
import pickle
import itertools
from tqdm import tqdm
import pandas as pd

###Creating dataset

In [ ]:
with open("Nlp HW2/corpus.txt","rb") as f:
  corpo = pickle.load(f)

corpo

In [24]:
#the corpus used for this task comes from hazm word list
corpus = list(set(corpo))
len(corpus)

35316

In [25]:
#roles contains all the possible roles in our dictionary
roles = [x[2] for x in hazm.words_list()]
roles = list(set(itertools.chain(*roles)))
roles

['N',
 'COMP',
 'POSTP',
 'V',
 'P',
 'PL',
 'NIN',
 'NUM',
 'INT',
 'AJC',
 'PR',
 'ADV',
 'DET',
 'CL',
 'CONJ',
 'PRO',
 'RES',
 'PS',
 'ZVR',
 'AJ']

In [29]:
lemmatizer = hazm.Lemmatizer()
stemmer = hazm.Stemmer()

In [36]:
'''this way of defining stop_words may not be usual,
but the main purpose has been to eliminate disturbing words;
such as: punctiation marks, pronouns and Persian\Arabic grammatical signs'''

add_list = ["ند","یم","ید","ی","تان","ش","ت","شان","مان","م","‍ها","ات","ان","ین","ون"]
stop_words = add_list
stop_words.extend([".","؛",",",":","?","!","؟","ْ","ٌ","ٍ","ً","ُ","ِ","َ","ّ","}","ؤ","{","»","«","ٰ","‌‌‌‌‌‌‌‌ٔ","ء","\""])

In [30]:
corpu = []
for word in corpus:
  corpu.append(word)
  corpu.append(lemmatizer.lemmatize(word))
  for ele in add_list:
    corpu.append(word+ele)

len(corpu)

600372

##Main code

###Config

In [22]:
#nlp is the dadmatools' pipeline used for lemmatization purpose only
pips = 'lem' 
nlp = language.Pipeline(pips)

print(nlp.analyze_pipes(pretty=True))


============================= Pipeline Overview =============================

#   Component   Assigns       Requires   Scores   Retokenizes
-   ---------   -----------   --------   ------   -----------
0   tokenizer                                     True       
                                                             
1   lemmatize   token.lemma                       False      

✔ No problems found.
{'summary': {'tokenizer': {'assigns': [], 'requires': [], 'scores': [], 'retokenizes': True}, 'lemmatize': {'assigns': ['token.lemma'], 'requires': [], 'scores': [], 'retokenizes': False}}, 'problems': {'tokenizer': [], 'lemmatize': []}, 'attrs': {'token.lemma': {'assigns': ['lemmatize'], 'requires': []}}}


In [37]:
#we use an args dictionary to initialize our model
args = {
    'corpus':corpus,
    'pipe':nlp,
    'stop_words':stop_words,
    'normalizer':hazm.Normalizer(),
    'lemmatizer':hazm.Lemmatizer(),
    'POS_tagger':hazm.POSTagger(model='Nlp HW2/resources/postagger.model'),
    'translator':None,
    'word_tokenizer':hazm.WordTokenizer(),
    'sent_tokenizer':hazm.SentenceTokenizer()
}

###Model

In [66]:
#as the name suggests, the class is used to spot and show the positin of foreign words within the text

class foreign_word_detector():

  def __init__(
      self,
      args
      ):
    
    self.corpus = args['corpus']
    self.stop_words = args['stop_words']
    self.lang_pipe = args['pipe']
    self.normalizer = args['normalizer']
    self.lemmatizer = args['lemmatizer']
    self.translator = args['translator']
    self.word_tokenizer = args['word_tokenizer']
    self.sent_tokenizer = args['sent_tokenizer']
    self.tagger = args['POS_tagger']

#the following function is the main funtion of the model used for detecting the foreign words and their position
  def detect(self, text):
    sentenced = self.sent_tokenizer.tokenize(text)

    normalized = [self.normalizer.normalize(sent) for sent in sentenced]
    normalized = [sent.replace("\u200c"," ") for sent in normalized]

    lemmed = [str(self.lang_pipe(sent)) for sent in normalized]

    words_list = [self.word_tokenizer.tokenize(sent) for sent in lemmed]

    final_words_list = []
    for sent in words_list:
      words_second_list = []
      tags = self.tagger.tag(sent)
      for i in range(len(tags)):
        if tags[i][1] != "V":
          words_second_list.append(tags[i][0])
      final_words_list.append(words_second_list)

    words_list = list(itertools.chain(*final_words_list))
    words_list = [word for word in words_list if word not in self.stop_words]
    lemmed_words_list = [lemmatizer.lemmatize(word) for word in words_list]
    

    output = dict()
    for word in lemmed_words_list:
      if word not in self.corpus:
        output[word] = []
        add_ind = 0

        string = text
        while word in string:

          begin = string.find(word)
          output[word].append((begin+add_ind,add_ind+begin+len(word)))

          string = string[begin+len(word):]
          add_ind+= begin+len(word)
    
    return output
          

###Test

####results

In [ ]:
test_data = pd.load_csv("Nlp HW2/test.csv")

####user run

In [64]:
#the run function, is an interactive way to map user_input to the detectors answer
def run():
  detector = foreign_word_detector(args)

  print("enter your sample text.\nenter an empty string if you want to end the process")
  while True : 
    text = input()
    if text == "": 
      break
    
    output = detector.detect(text)
    print(output)


In [67]:
run()

enter your sample text.
enter an empty string if you want to end the process
سیستم کامپیوتر من خراب است؛ سیستم کامپیوتر شما چطور؟
{'سیستم': [(0, 5), (28, 33)], 'کامپیوتر': [(6, 14), (34, 42)]}



##New idea

In [7]:
#We import an English dictionary. Add the english words in "words" and the corresponding phonetics to "phonics_list".
words = []
phonics_list = []
dict_file = open('Nlp HW2/cmudict.dict', 'r')
with dict_file as f:
    phonics = [line.rstrip('\n') for line in f]
    for p in phonics:
        x = p.split(' ')
        words.append(x[0])
        phonics_list.append(' '.join(x[1:]))

In [8]:
phons_unique = list(set(itertools.chain(*[ph.split(" ") for ph in phonics_list])))
phons_as_list = [ph.split(" ") for ph in phonics_list]

In [10]:
with open("Nlp HW2/phons_exchange.txt","rb") as file:
  phons_dict = pickle.load(file)

In [69]:
phons_dict["NG"] = ("ن")
phons_dict["TH"] = ("ت")
phons_dict["AO2"] = phons_dict["AO1"]
phons_dict["IH2"] = ("ای","ی")
phons_dict["IH0"] = phons_dict["IH2"]
phons_dict["IH1"] = phons_dict["IH0"]

In [70]:
#Converting our words to their persian correspondent with the use of phonetics in the "persianized"
persianized_list = []
persianized = []
for x in phons_as_list:
  word_list = []
  count=0
  for ph in x:
    count+=1
    if len(phons_dict[ph])>1:
      if count==1:
        word_list.append(phons_dict[ph][0])
      else:
        word_list.append(phons_dict[ph][1])
    else:
      word_list.append(phons_dict[ph][0])

  persianized_list.append(word_list)
  persianized.append(''.join(word_list))

In [15]:
#Checking some words from "words"
words[41000:41010]

['fancier',
 'fanciers',
 'fancies',
 'fanciest',
 'fanciful',
 'fancy',
 'fandango',
 'fandel',
 'fandrich',
 'fane']

In [16]:
#Checking the corresponding persionized words from "persionized"
persianized[41000:41010]

['فنسير',
 'فنسيرز',
 'فنسيز',
 'فنسيست',
 'فنسیفل',
 'فنسي',
 'فندنگو',
 'فندل',
 'فندریک',
 'فین']

In [71]:
#Checking if our persionized method works fine on some words.
listt = ['contribution','thanks','computer','system','task','time','garden','burger','chop','fandrich','infinity']

for i in range(len(words)):
  if words[i] in listt:
    print(words[i])
    print(phons_as_list[i])
    print(i)
    print(persianized[i])
    print("-----------------")

burger
['B', 'ER1', 'G', 'ER0']
16631
برگر
-----------------
chop
['CH', 'AA1', 'P']
21283
چاپ
-----------------
computer
['K', 'AH0', 'M', 'P', 'Y', 'UW1', 'T', 'ER0']
23974
کمپیوتر
-----------------
contribution
['K', 'AA2', 'N', 'T', 'R', 'AH0', 'B', 'Y', 'UW1', 'SH', 'AH0', 'N']
25006
کانتربیوشن
-----------------
fandrich
['F', 'AE1', 'N', 'D', 'R', 'IH0', 'K']
41008
فندریک
-----------------
garden
['G', 'AA1', 'R', 'D', 'AH0', 'N']
46458
گاردن
-----------------
infinity
['IH2', 'N', 'F', 'IH1', 'N', 'IH0', 'T', 'IY0']
59556
اینفینیتي
-----------------
system
['S', 'IH1', 'S', 'T', 'AH0', 'M']
118902
سیستم
-----------------
task
['T', 'AE1', 'S', 'K']
119638
تسک
-----------------
thanks
['TH', 'AE1', 'NG', 'K', 'S']
120780
تنکس
-----------------
time
['T', 'AY1', 'M']
121710
تایم
-----------------


In [45]:
#phons_dict contains the Persian equivalent of English phonetics
phons_dict

{'EH0': ('ا', ''),
 'ER2': ('ار', 'ر'),
 'old': ('',),
 'EY0': ('ای', 'ی'),
 'IY2': ('ی',),
 'name': ('',),
 'S': ('س',),
 'CH': ('چ',),
 'K': ('ک',),
 'UW2': ('یو',),
 'AA0': ('آ', 'ا'),
 'AY2': ('آی', 'ای'),
 'OY0': ('اوی', 'وی'),
 'AA1': ('آ', 'ا'),
 'L': ('ل',),
 'UH0': ('و',),
 'B': ('ب',),
 'F': ('ف',),
 'OY1': ('اوی', 'وی'),
 'abbrev': ('',),
 'UH2': ('و',),
 'D': ('د',),
 'JH': ('ج',),
 'W': ('و',),
 'G': ('گ',),
 'IH0': ('ی',),
 'UH1': ('و',),
 'french': ('',),
 'Z': ('ز',),
 'AA2': ('آ', 'ا'),
 'OY2': ('اوی', 'وی'),
 'SH': ('ش',),
 'AH0': ('ا', ''),
 'foreign': ('',),
 'UW0': ('یو',),
 'AH1': ('ا', ''),
 'IH1': ('ی',),
 'EY2': ('ای', 'ی'),
 'AH2': ('ا', ''),
 'OW2': ('او', 'و'),
 'Y': ('ی',),
 'AE2': ('',),
 'DH': ('ض',),
 'P': ('پ',),
 'ER1': ('ار', 'ر'),
 'AW2': ('او',),
 '#': ('',),
 'EH1': ('ا', ''),
 'AE1': ('',),
 'EH2': ('ا', ''),
 'N': ('ن',),
 'ZH': ('ز',),
 'OW1': ('او', 'و'),
 'UW1': ('او', 'و'),
 'AY0': ('آی', 'ای'),
 'AY1': ('آی', 'ای'),
 'AE0': ('',),
 'NG': 'ن'